In [13]:
import pandas as pd


In [14]:
#loading the first dataset
first_data = pd.read_csv("First.csv")
first_data.head()

,No,Date/Time,Date/time end,Altitude [m],Size fraction,Mass v [µg/m**3],Na+ [µg/m**3],[NH4]+ [µg/m**3],K+ [µg/m**3],Mg2+ [µg/m**3],...,Cu [ng/m**3],Zn [ng/m**3],Pb [ng/m**3],As [ng/m**3],Se [ng/m**3],Sr [ng/m**3],Rb [ng/m**3],Ba [ng/m**3],La [ng/m**3],Ce [ng/m**3]
0,866,2010-01-11T16:30:00,2010-01-12T17:16,42,0.05 - 0.14 µm,0.063,0.0026,0.0061,<0.0006,<0.0006,...,0.17,0.21,<0.0228,0.003,<0.001,<0.0372,<0.0048,<0.0048,0.006,<0.006
1,867,2010-01-11T16:30:00,2010-01-12T17:16,42,0.14 - 0.42 µm,0.636,0.0661,0.0119,<0.0006,0.0056,...,0.16,0.09,0.026,0.025,0.002,0.051,0.011,<0.0048,0.057,<0.006
2,868,2010-01-11T16:30:00,2010-01-12T17:16,42,0.42 - 1.2 µm,5.886,0.6887,0.0016,0.02,0.0922,...,0.16,0.24,0.052,0.02,0.041,0.831,0.064,1.016,0.682,0.068
3,869,2010-01-11T16:30:00,2010-01-12T17:16,42,1.2 - 3.5 µm,28.971,6.7001,0.0021,0.2373,1.0962,...,0.34,0.43,0.055,0.168,0.096,3.35,0.331,4.086,2.026,0.318
4,870,2010-01-11T16:30:00,2010-01-12T17:16,42,3.5 - 10 µm,31.786,6.8745,0.0109,0.3071,1.1,...,0.25,0.41,0.063,0.079,0.004,4.179,0.425,1.977,1.578,0.562


In [15]:
#loading the second dataset
second_data = pd.read_csv("Second.csv")
second_data.head()

,No,Date/Time,Date/time end,Altitude [m],Size fraction,Mass v [µg/m**3],Na+ [µg/m**3],[NH4]+ [µg/m**3],K+ [µg/m**3],Mg2+ [µg/m**3],...,Cu [ng/m**3],Zn [ng/m**3],Pb [ng/m**3],As [ng/m**3],Se [ng/m**3],Sr [ng/m**3],Rb [ng/m**3],Ba [ng/m**3],La [ng/m**3],Ce [ng/m**3]
0,821,2011-01-11T14:48:00,2011-01-14T11:28,42,0.05 - 0.14 µm,0.477,-0.0004,0.0083,<0.0006,0.0007,...,0.08,0.38,0.023,0.005,0.001,0.03,<0.0048,<0.0048,<0.0036,0.01
1,822,2011-01-11T14:48:00,2011-01-14T11:28,42,0.14 - 0.42 µm,1.617,0.0094,0.0892,0.0044,0.005,...,0.23,0.60,0.213,0.041,0.013,0.13,0.012,0.22,0.06,0.03
2,823,2011-01-11T14:48:00,2011-01-14T11:28,42,0.42 - 1.2 µm,6.705,0.1471,0.0130,0.0128,0.0164,...,0.29,0.62,0.249,0.036,0.016,0.41,0.034,0.59,<0.0036,0.08
3,824,2011-01-11T14:48:00,2011-01-14T11:28,42,1.2 - 3.5 µm,40.365,1.2907,0.0140,0.0629,0.0765,...,0.4,1.18,0.948,<0.003,0.082,6.45,1.294,5.9,4.05,1.02
4,825,2011-01-11T14:48:00,2011-01-14T11:28,42,3.5 - 10 µm,32.766,1.8802,0.0111,0.0736,0.084,...,0.43,1.03,0.983,<0.003,NaN,5.37,1.299,7.31,4.4,1.08


In [16]:
#loading the third dataset
third_data = pd.read_csv("Third.csv")
third_data.head()

,No,Date/Time,Date/time end,Altitude [m],Size fraction,Mass v [µg/m**3],Na+ [µg/m**3],[NH4]+ [µg/m**3],K+ [µg/m**3],Mg2+ [µg/m**3],...,Cu [ng/m**3],Zn [ng/m**3],Pb [ng/m**3],As [ng/m**3],Se [ng/m**3],Sr [ng/m**3],Rb [ng/m**3],Ba [ng/m**3],La [ng/m**3],Ce [ng/m**3]
0,1251,2012-01-13T11:31:00,2012-01-12T15:10,42,0.05 - 0.14 µm,0.266,0.1013,0.0105,<0.0006,0.0029,...,<0.048,0.166,0.061,0.008,<0.001,0.077,0.012,<0.0048,<0.0036,<0.006
1,1252,2012-01-13T11:31:00,2012-01-12T15:10,42,0.14 - 0.42 µm,1.144,0.0107,0.0353,0.0036,0.003,...,0.055,0.295,0.192,0.199,0.015,0.3,0.024,0.321,0.11,0.028
2,1253,2012-01-13T11:31:00,2012-01-12T15:10,42,0.42 - 1.2 µm,8.953,0.1442,0.0041,0.0106,0.0231,...,0.125,0.447,0.385,0.206,0.145,2.934,0.118,4.167,0.127,0.44
3,1254,2012-01-13T11:31:00,2012-01-12T15:10,42,1.2 - 3.5 µm,44.887,1.199,0.0176,0.0769,0.1043,...,0.573,1.354,0.638,1.693,0.2,12.391,0.704,10.299,1.476,2.64
4,1255,2012-01-13T11:31:00,2012-01-12T15:10,42,3.5 - 10 µm,40.703,1.4175,0.0131,0.0795,0.1141,...,1.18,3.176,1.968,0.606,0.011,15.973,3.336,24.78,14.877,4.332


In [17]:
#Merging the three datasets
data = pd.concat([first_data, second_data, third_data], ignore_index=True)
data.head()

,No,Date/Time,Date/time end,Altitude [m],Size fraction,Mass v [µg/m**3],Na+ [µg/m**3],[NH4]+ [µg/m**3],K+ [µg/m**3],Mg2+ [µg/m**3],...,Cu [ng/m**3],Zn [ng/m**3],Pb [ng/m**3],As [ng/m**3],Se [ng/m**3],Sr [ng/m**3],Rb [ng/m**3],Ba [ng/m**3],La [ng/m**3],Ce [ng/m**3]
0,866,2010-01-11T16:30:00,2010-01-12T17:16,42,0.05 - 0.14 µm,0.063,0.0026,0.0061,<0.0006,<0.0006,...,0.17,0.21,<0.0228,0.003,<0.001,<0.0372,<0.0048,<0.0048,0.006,<0.006
1,867,2010-01-11T16:30:00,2010-01-12T17:16,42,0.14 - 0.42 µm,0.636,0.0661,0.0119,<0.0006,0.0056,...,0.16,0.09,0.026,0.025,0.002,0.051,0.011,<0.0048,0.057,<0.006
2,868,2010-01-11T16:30:00,2010-01-12T17:16,42,0.42 - 1.2 µm,5.886,0.6887,0.0016,0.02,0.0922,...,0.16,0.24,0.052,0.02,0.041,0.831,0.064,1.016,0.682,0.068
3,869,2010-01-11T16:30:00,2010-01-12T17:16,42,1.2 - 3.5 µm,28.971,6.7001,0.0021,0.2373,1.0962,...,0.34,0.43,0.055,0.168,0.096,3.35,0.331,4.086,2.026,0.318
4,870,2010-01-11T16:30:00,2010-01-12T17:16,42,3.5 - 10 µm,31.786,6.8745,0.0109,0.3071,1.1,...,0.25,0.41,0.063,0.079,0.004,4.179,0.425,1.977,1.578,0.562


In [18]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 655 entries, 0 to 654
Data columns (total 36 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   No                  655 non-null    int64  
 1   Date/Time           655 non-null    object 
 2   Date/time end       655 non-null    object 
 3   Altitude [m]        655 non-null    int64  
 4   Size fraction       655 non-null    object 
 5   Mass v [µg/m**3]    655 non-null    float64
 6   Na+ [µg/m**3]       655 non-null    object 
 7   [NH4]+ [µg/m**3]    655 non-null    object 
 8   K+ [µg/m**3]        655 non-null    object 
 9   Mg2+ [µg/m**3]      655 non-null    object 
 10  Ca2+ [µg/m**3]      655 non-null    object 
 11  Cl- [µg/m**3]       655 non-null    float64
 12  [NO3]- [µg/m**3]    655 non-null    object 
 13  [SO4]2- [µg/m**3]   655 non-null    float64
 14  [C2O4]2- [µg/m**3]  655 non-null    object 
 15  Br- [µg/m**3]       655 non-null    object 
 16  C org [µ

In [19]:
#checking missing values
print(data.isna().sum())
print("number of rows with missing values :", data.isna().any(axis=1).sum())

No                     0
Date/Time              0
Date/time end          0
Altitude [m]           0
Size fraction          0
Mass v [µg/m**3]       0
Na+ [µg/m**3]          0
[NH4]+ [µg/m**3]       0
K+ [µg/m**3]           0
Mg2+ [µg/m**3]         0
Ca2+ [µg/m**3]         0
Cl- [µg/m**3]          0
[NO3]- [µg/m**3]       0
[SO4]2- [µg/m**3]      0
[C2O4]2- [µg/m**3]     0
Br- [µg/m**3]          0
C org [µg/m**3]        3
EC [µg/m**3]           2
TC [µg/m**3]           1
Ca [ng/m**3]           8
Ti [ng/m**3]           8
V [ng/m**3]            8
Cr [ng/m**3]           8
Mn [ng/m**3]           8
Fe [ng/m**3]           8
Ni [ng/m**3]           8
Cu [ng/m**3]           8
Zn [ng/m**3]           8
Pb [ng/m**3]           9
As [ng/m**3]          10
Se [ng/m**3]          12
Sr [ng/m**3]           8
Rb [ng/m**3]           8
Ba [ng/m**3]          11
La [ng/m**3]          12
Ce [ng/m**3]           9
dtype: int64
number of rows with missing values : 26


In [20]:
#dropping missing values
data = data.dropna()
data.shape

(629, 36)

In [21]:
#checking the datetime data type 11:47:08 
Format = "%Y-%m-%dT%H:%M:%S"  

data[['Date/Time', 'Date/time end']] = data[['Date/Time', 'Date/time end']].apply(lambda x: x.str.strip())
# Preprocess 'Date/time end' to add seconds if missing
data['Date/time end'] = data['Date/time end'].apply(lambda x: x + ':00' if len(x) == 16 else x)
data['Date/Time'] = pd.to_datetime(data['Date/Time'], format=Format)
data['Date/time end'] = pd.to_datetime(data['Date/time end'],format=Format )

In [22]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 629 entries, 0 to 654
Data columns (total 36 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   No                  629 non-null    int64         
 1   Date/Time           629 non-null    datetime64[ns]
 2   Date/time end       629 non-null    datetime64[ns]
 3   Altitude [m]        629 non-null    int64         
 4   Size fraction       629 non-null    object        
 5   Mass v [µg/m**3]    629 non-null    float64       
 6   Na+ [µg/m**3]       629 non-null    object        
 7   [NH4]+ [µg/m**3]    629 non-null    object        
 8   K+ [µg/m**3]        629 non-null    object        
 9   Mg2+ [µg/m**3]      629 non-null    object        
 10  Ca2+ [µg/m**3]      629 non-null    object        
 11  Cl- [µg/m**3]       629 non-null    float64       
 12  [NO3]- [µg/m**3]    629 non-null    object        
 13  [SO4]2- [µg/m**3]   629 non-null    float64       
 14 

In [23]:

str_data = data.astype(str)

# Create mask for negative values (strings that start with "-")
mask_neg = str_data.applymap(lambda x: x.strip().startswith("-"))

# Replace negative values with 0
data[mask_neg] = 0

C:\Users\Tom\AppData\Local\Temp\ipykernel_14536\1106189905.py:4: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  mask_neg = str_data.applymap(lambda x: x.strip().startswith("-"))


In [24]:
#Saving the dataset 
data.to_csv("data.csv")